# A notebook used to learn how to create a dataframe of a LoL users games.

In [1]:
# Import the needed librariers.
# Riot watcher is a wrapper for the LoL Api to make things easier
from riotwatcher import LolWatcher, ApiError
import pandas as pd
import numpy as np
import datetime
import yaml

In [2]:
# Here we store the password for the api.
with open("Secrets.yml", 'r') as ymlfile:
    cfg = yaml.safe_load(ymlfile)

password = cfg['access_code']

In [3]:
# This is some set up for the riot watcher library
lol_watcher = LolWatcher(password)
my_region = 'na1'

In [4]:
# Returns info on a summoner by name
user = lol_watcher.summoner.by_name(my_region, 'HungryHipno')
user

{'id': 'XFnQbQ_4AF6L-z131nFHKWFvqdQ8rFk0PIZsh8kiaJmGpUQ',
 'accountId': 'rBhsls_tfYDCqo51hTN0icB8AAgKMVIWNQQS5W7pEj6_HGk',
 'puuid': 'hLmcnhrJXFSXtPdCqnp9kPr7D2FMyIhgpRsF9gI44PwLQXkldjztf1kic5dQMtUM44K4uoTKDqpdWA',
 'name': 'HungryHipNo',
 'profileIconId': 4569,
 'revisionDate': 1593054013000,
 'summonerLevel': 310}

## Here is where a list of all the matches are collected and prepared.

In [5]:
# Returns a dict of the last 100 games played by this account.
# Regardless how long ago the first of these 100 mactches is.
tempTimeline = lol_watcher.match.matchlist_by_account(my_region, user['accountId'])

In [6]:
# Convert dict to a dataframe and look at the data.
dfTemp = pd.DataFrame(tempTimeline['matches'])
dfTemp['date'] = pd.to_datetime(dfTemp['timestamp'], unit='ms')

In [7]:
dfTemp

,platformId,gameId,champion,queue,season,timestamp,role,lane,date
0,NA1,3473010118,63,450,13,1593052376999,DUO_SUPPORT,MID,2020-06-25 02:32:56.999
1,NA1,3472927430,81,450,13,1593051298569,DUO_SUPPORT,NONE,2020-06-25 02:14:58.569
2,NA1,3472913199,41,450,13,1593049559586,DUO,MID,2020-06-25 01:45:59.586
3,NA1,3472879275,41,450,13,1593048098285,DUO_SUPPORT,NONE,2020-06-25 01:21:38.285
4,NA1,3472854527,89,450,13,1593045868874,DUO_SUPPORT,TOP,2020-06-25 00:44:28.874
...,...,...,...,...,...,...,...,...,...
95,NA1,3453016413,75,420,13,1591572050660,SOLO,TOP,2020-06-07 23:20:50.660
96,NA1,3452991704,17,420,13,1591569787812,DUO,MID,2020-06-07 22:43:07.812
97,NA1,3452888068,75,420,13,1591567369788,SOLO,TOP,2020-06-07 22:02:49.788
98,NA1,3452922398,122,420,13,1591565605647,SOLO,TOP,2020-06-07 21:33:25.647


In [8]:
# This creates a list of every game played by the user since the chosen firstday.

# 1514782800000 = Mon Jan 01 2018 00:00:00 in Unix Milliseconds
firstDay = 1514782800000

# This finds out the minimum timestamp from the first api call and starts there.
# It also converts the variable from a df data type to a python int
minDay = np.int64(dfTemp['timestamp'].min()).item() -1
while minDay > firstDay:
    try:
        # The api only allows you to search by day if youre search period is a week or less.
        # So this loop goes through every week from the earliest of match of the first query 
        tempTimeline = lol_watcher.match.matchlist_by_account(my_region, 
                                                          'rBhsls_tfYDCqo51hTN0icB8AAgKMVIWNQQS5W7pEj6_HGk', 
                                                          begin_time=minDay-604800000, 
                                                          end_time = minDay)
        minDay-=604800000
        dfTemp = dfTemp.append(tempTimeline['matches'])
        print(tempTimeline['matches'][0])
    except ApiError as err:
        # If there is no data for a week we recieve a 404 error.
        print(err.response.status_code)
        minDay-=604800000


{'platformId': 'NA1', 'gameId': 3452822485, 'champion': 350, 'queue': 420, 'season': 13, 'timestamp': 1591560248727, 'role': 'DUO_SUPPORT', 'lane': 'BOTTOM'}
{'platformId': 'NA1', 'gameId': 3444222252, 'champion': 15, 'queue': 420, 'season': 13, 'timestamp': 1590952630218, 'role': 'DUO_CARRY', 'lane': 'BOTTOM'}
{'platformId': 'NA1', 'gameId': 3433823896, 'champion': 235, 'queue': 900, 'season': 13, 'timestamp': 1590289801975, 'role': 'NONE', 'lane': 'JUNGLE'}
{'platformId': 'NA1', 'gameId': 3424816324, 'champion': 31, 'queue': 900, 'season': 13, 'timestamp': 1589746386925, 'role': 'DUO_CARRY', 'lane': 'TOP'}
{'platformId': 'NA1', 'gameId': 3412421138, 'champion': 33, 'queue': 420, 'season': 13, 'timestamp': 1589015207844, 'role': 'NONE', 'lane': 'JUNGLE'}
{'platformId': 'NA1', 'gameId': 3403887712, 'champion': 17, 'queue': 420, 'season': 13, 'timestamp': 1588498759003, 'role': 'SOLO', 'lane': 'TOP'}
{'platformId': 'NA1', 'gameId': 3393439080, 'champion': 111, 'queue': 420, 'season': 13

In [9]:
# Convert the unix millis to a new column of day time to make it more readable
dfTemp['date'] = pd.to_datetime(dfTemp['timestamp'], unit='ms')
dfTemp.sort_values(by=['timestamp'], inplace=True)
dfTemp.reset_index(drop=True, inplace=True)

In [10]:
dfTemp

,platformId,gameId,champion,queue,season,timestamp,role,lane,date
0,NA1,2812808319,63,420,11,1530040714666,DUO_SUPPORT,BOTTOM,2018-06-26 19:18:34.666
1,NA1,2813096089,223,420,11,1530066099722,DUO_SUPPORT,BOTTOM,2018-06-27 02:21:39.722
2,NA1,2813203360,63,420,11,1530068223875,DUO_SUPPORT,BOTTOM,2018-06-27 02:57:03.875
3,NA1,2813333414,16,420,11,1530071249610,DUO_SUPPORT,BOTTOM,2018-06-27 03:47:29.610
4,NA1,2814277304,25,900,11,1530157073729,NONE,JUNGLE,2018-06-28 03:37:53.729
...,...,...,...,...,...,...,...,...,...
2919,NA1,3472854527,89,450,13,1593045868874,DUO_SUPPORT,TOP,2020-06-25 00:44:28.874
2920,NA1,3472879275,41,450,13,1593048098285,DUO_SUPPORT,NONE,2020-06-25 01:21:38.285
2921,NA1,3472913199,41,450,13,1593049559586,DUO,MID,2020-06-25 01:45:59.586
2922,NA1,3472927430,81,450,13,1593051298569,DUO_SUPPORT,NONE,2020-06-25 02:14:58.569


In [11]:
#Check for any duplicates in DF and remove them
duplicateRowsDF = dfTemp[dfTemp.duplicated()]
print(duplicateRowsDF)

Empty DataFrame
Columns: [platformId, gameId, champion, queue, season, timestamp, role, lane, date]
Index: []


## Here is where there is a dive into the data.

In [12]:
# This creates a new dataframe of only Aram games.
dfAram = dfTemp.loc[(dfTemp['queue'] == 450)]

In [13]:
dfAram

,platformId,gameId,champion,queue,season,timestamp,role,lane,date
205,NA1,2861921362,91,450,11,1536467521065,DUO_SUPPORT,MID,2018-09-09 04:32:01.065
206,NA1,2861935848,163,450,11,1536468926806,DUO_SUPPORT,TOP,2018-09-09 04:55:26.806
207,NA1,2861961065,10,450,11,1536470482712,DUO,NONE,2018-09-09 05:21:22.712
349,NA1,2883238948,203,450,11,1539028088045,DUO_SUPPORT,TOP,2018-10-08 19:48:08.045
362,NA1,2885159375,54,450,11,1539295170450,DUO_SUPPORT,TOP,2018-10-11 21:59:30.450
...,...,...,...,...,...,...,...,...,...
2919,NA1,3472854527,89,450,13,1593045868874,DUO_SUPPORT,TOP,2020-06-25 00:44:28.874
2920,NA1,3472879275,41,450,13,1593048098285,DUO_SUPPORT,NONE,2020-06-25 01:21:38.285
2921,NA1,3472913199,41,450,13,1593049559586,DUO,MID,2020-06-25 01:45:59.586
2922,NA1,3472927430,81,450,13,1593051298569,DUO_SUPPORT,NONE,2020-06-25 02:14:58.569


In [14]:
# Here we make a call to the api to get info about a single game.
game = lol_watcher.match.by_id(my_region,'2861921362')

In [15]:
for data in game:
    print(data, game[data])
    print()

gameId 2861921362

platformId NA1

gameCreation 1536467521065

gameDuration 1211

queueId 450

mapId 12

seasonId 11

gameVersion 8.17.243.3340

gameMode ARAM

gameType MATCHED_GAME

teams [{'teamId': 100, 'win': 'Win', 'firstBlood': False, 'firstTower': False, 'firstInhibitor': True, 'firstBaron': False, 'firstDragon': False, 'firstRiftHerald': False, 'towerKills': 3, 'inhibitorKills': 1, 'baronKills': 0, 'dragonKills': 0, 'vilemawKills': 0, 'riftHeraldKills': 0, 'dominionVictoryScore': 0, 'bans': []}, {'teamId': 200, 'win': 'Fail', 'firstBlood': True, 'firstTower': True, 'firstInhibitor': False, 'firstBaron': False, 'firstDragon': False, 'firstRiftHerald': False, 'towerKills': 2, 'inhibitorKills': 0, 'baronKills': 0, 'dragonKills': 0, 'vilemawKills': 0, 'riftHeraldKills': 0, 'dominionVictoryScore': 0, 'bans': []}]

participants [{'participantId': 1, 'teamId': 100, 'championId': 427, 'spell1Id': 4, 'spell2Id': 7, 'highestAchievedSeasonTier': 'BRONZE', 'stats': {'participantId': 1, 'wi

In [16]:
for team in game['teams']:
    print(team)
    print()

{'teamId': 100, 'win': 'Win', 'firstBlood': False, 'firstTower': False, 'firstInhibitor': True, 'firstBaron': False, 'firstDragon': False, 'firstRiftHerald': False, 'towerKills': 3, 'inhibitorKills': 1, 'baronKills': 0, 'dragonKills': 0, 'vilemawKills': 0, 'riftHeraldKills': 0, 'dominionVictoryScore': 0, 'bans': []}

{'teamId': 200, 'win': 'Fail', 'firstBlood': True, 'firstTower': True, 'firstInhibitor': False, 'firstBaron': False, 'firstDragon': False, 'firstRiftHerald': False, 'towerKills': 2, 'inhibitorKills': 0, 'baronKills': 0, 'dragonKills': 0, 'vilemawKills': 0, 'riftHeraldKills': 0, 'dominionVictoryScore': 0, 'bans': []}



In [17]:

for data in game['participants']:
    print(data)
    print()

{'participantId': 1, 'teamId': 100, 'championId': 427, 'spell1Id': 4, 'spell2Id': 7, 'highestAchievedSeasonTier': 'BRONZE', 'stats': {'participantId': 1, 'win': True, 'item0': 3094, 'item1': 3046, 'item2': 3095, 'item3': 1043, 'item4': 1037, 'item5': 1052, 'item6': 2052, 'kills': 5, 'deaths': 8, 'assists': 12, 'largestKillingSpree': 2, 'largestMultiKill': 1, 'killingSprees': 1, 'longestTimeSpentLiving': 164, 'doubleKills': 0, 'tripleKills': 0, 'quadraKills': 0, 'pentaKills': 0, 'unrealKills': 0, 'totalDamageDealt': 42873, 'magicDamageDealt': 14228, 'physicalDamageDealt': 28645, 'trueDamageDealt': 0, 'largestCriticalStrike': 336, 'totalDamageDealtToChampions': 9231, 'magicDamageDealtToChampions': 3512, 'physicalDamageDealtToChampions': 5719, 'trueDamageDealtToChampions': 0, 'totalHeal': 1879, 'totalUnitsHealed': 3, 'damageSelfMitigated': 9423, 'damageDealtToObjectives': 4371, 'damageDealtToTurrets': 4371, 'visionScore': 0, 'timeCCingOthers': 7, 'totalDamageTaken': 16240, 'magicalDamageT

In [18]:
for data in game['participants'][0]:
    print(data, game['participants'][0][data])
    print()

participantId 1

teamId 100

championId 427

spell1Id 4

spell2Id 7

highestAchievedSeasonTier BRONZE

stats {'participantId': 1, 'win': True, 'item0': 3094, 'item1': 3046, 'item2': 3095, 'item3': 1043, 'item4': 1037, 'item5': 1052, 'item6': 2052, 'kills': 5, 'deaths': 8, 'assists': 12, 'largestKillingSpree': 2, 'largestMultiKill': 1, 'killingSprees': 1, 'longestTimeSpentLiving': 164, 'doubleKills': 0, 'tripleKills': 0, 'quadraKills': 0, 'pentaKills': 0, 'unrealKills': 0, 'totalDamageDealt': 42873, 'magicDamageDealt': 14228, 'physicalDamageDealt': 28645, 'trueDamageDealt': 0, 'largestCriticalStrike': 336, 'totalDamageDealtToChampions': 9231, 'magicDamageDealtToChampions': 3512, 'physicalDamageDealtToChampions': 5719, 'trueDamageDealtToChampions': 0, 'totalHeal': 1879, 'totalUnitsHealed': 3, 'damageSelfMitigated': 9423, 'damageDealtToObjectives': 4371, 'damageDealtToTurrets': 4371, 'visionScore': 0, 'timeCCingOthers': 7, 'totalDamageTaken': 16240, 'magicalDamageTaken': 2963, 'physicalDa

In [19]:
game['participantIdentities']

[{'participantId': 1,
  'player': {'platformId': 'NA1',
   'accountId': '0OD3ymOx_fJvNuWfXvC7Duqf8ibgnJF9Kuu6-K7n423xqds',
   'summonerName': 'manoakane',
   'summonerId': 'T51LDIUlbWnekjaEat_RL1phPubWSKER5zoCPGN9Oq0HCDo',
   'currentPlatformId': 'NA1',
   'currentAccountId': '0OD3ymOx_fJvNuWfXvC7Duqf8ibgnJF9Kuu6-K7n423xqds',
   'matchHistoryUri': '/v1/stats/player_history/NA1/237022457',
   'profileIcon': 1634}},
 {'participantId': 2,
  'player': {'platformId': 'NA1',
   'accountId': 'QWVaSLUiJQVfxTEGmCKrNjLhV5zgGIACtHEtjhY0LPIOpZs',
   'summonerName': 'Yung Dicc',
   'summonerId': 'td_ymJyN-18Pu4L83dU3sS2DZbgkZhDJ4ozXKd5x_fdwVjE',
   'currentPlatformId': 'NA1',
   'currentAccountId': 'QWVaSLUiJQVfxTEGmCKrNjLhV5zgGIACtHEtjhY0LPIOpZs',
   'matchHistoryUri': '/v1/stats/player_history/NA1/241536442',
   'profileIcon': 3560}},
 {'participantId': 3,
  'player': {'platformId': 'NA1',
   'accountId': 'GjEY9lItbVLvXEXp2t6KS1QC9enR5zzZAYWIfJr509kYGIk',
   'summonerName': 'Dyeda',
   'summonerI

In [21]:
for data in game['participantIdentities'][0]['player']:
    print(data, game['participantIdentities'][0]['player'][data])
    print()

platformId NA1

accountId 0OD3ymOx_fJvNuWfXvC7Duqf8ibgnJF9Kuu6-K7n423xqds

summonerName manoakane

summonerId T51LDIUlbWnekjaEat_RL1phPubWSKER5zoCPGN9Oq0HCDo

currentPlatformId NA1

currentAccountId 0OD3ymOx_fJvNuWfXvC7Duqf8ibgnJF9Kuu6-K7n423xqds

matchHistoryUri /v1/stats/player_history/NA1/237022457

profileIcon 1634

